In [1]:
%matplotlib notebook
import os
import time
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from pprint import pprint
import matplotlib.colors as colors

# qcodes imports
import qcodes as qc
from qcodes import (Measurement,
                    experiments,
                    initialise_database,
                    initialise_or_create_database_at,
                    load_by_guid,
                    load_by_run_spec,
                    load_experiment,
                    load_last_experiment,
                    load_or_create_experiment,
                    new_experiment,
                    ManualParameter)
from qcodes.utils.dataset.doNd import do1d, do2d, do0d
from qcodes.dataset.plotting import plot_dataset
from qcodes.logger.logger import start_all_logging
from qcodes.tests.instrument_mocks import DummyInstrument, DummyInstrumentWithMeasurement
from qcodes.instrument.specialized_parameters import ElapsedTimeParameter
from qcodes.utils.validators import Numbers, Arrays
from qcodes.utils.metadata import diff_param_values
from qcodes.instrument.parameter import ParameterWithSetpoints, Parameter, ScaledParameter
from qcodes.interactive_widget import experiments_widget


# Import instrument drivers
#from qcodes.instrument_drivers.agilent.Agilent_34400A import Agilent_34400A # Agilent DMM 34411A
#from qcodes.instrument_drivers.Keysight.Keysight_34465A_submodules import Keysight_34465A # Keysight DMM 34465A
import zhinst.qcodes as ziqc # ZI MFLI lock-in amplifier
import nidaqmx
from qcodes_addons.NIDAQ import DAQAnalogInputs # NI DAQ
from qcodes_addons.SP927v1 import SP927 # Basel DAC
from qcodes_addons.Oxford_Instruments_IPS120 import OxfordInstruments_IPS120
#from ANC350 import ANC350 # attocube controller
#from ANC350Lib.v4 import ANC350v4Lib
#libfile = ANC350v4Lib(mydir+r"\anc350v4.dll")

# Import helper functions

from qcodes_addons.Datahelp import (dataset_to_numpy,
                      datasetID_to_numpy,
                      plot2D_by_ID,
                      define_detuning)
from qcodes_addons.Parameterhelp import (GateParameter,
                           VirtualGateParameter,
                           MultiDAQParameter,
                           AxisParameter,
                           ZILockinParameter)
from Measurementhelp import (do1d_time,
                             do2d_time,
                             do1d_realtime,
                             do1d_break,
                             do2d_break)

In [2]:
db_name = "Untitled.db" # Database name
sample_name = "Sample_name" # Sample name
exp_name = "Experiment_name" # Experiment name

db_file_path = os.path.join(os.getcwd(), db_name)
qc.config.core.db_location = db_file_path
initialise_or_create_database_at(db_file_path)

experiment = load_or_create_experiment(experiment_name = exp_name,
                                       sample_name = sample_name)

station = qc.Station()

In [3]:
# Live plotting
import IPython.lib.backgroundjobs as bg
from plottr.apps import inspectr

# Live plotting
jobs = bg.BackgroundJobManager()
jobs.new(inspectr.main, db_file_path)

<BackgroundJob #0: <function main at 0x00000259A3151C10>>

In [4]:
station = qc.Station()

#Adding setup devices

#DAC BASPI
dac = SP927('LNHR_dac', 'ASRL3::INSTR') 
station.add_component(dac)

#dac.close()
#station.remove_component('LNHR_dac')


#UHFLI Zurich Instruments 
uhfli = ziqc.UHFLI("ZI", "dev2405", interface="USB", host="127.0.0.1")
station.add_component(uhfli)




#NIDAQ National Instruments: NI USB-6363 DAQ
daq_chs = [0,1,2,3,4,5] 
# Specify the DAQ channels to be recorded
# Channel numbers as stated on the DAQ front panel
DAQ = DAQAnalogInputs(name ='daq',
                      dev_name = 'Dev2', # Device name can be found in NI-MAX
                      rate = 2000, 
                      chs = daq_chs,
                      int_time = 0.125)
station.add_component(DAQ)



#IPS120 Oxford Instruments (using RS232)
ips = OxfordInstruments_IPS120(name = "IPS", address = "ASRL4::INSTR", use_gpib=False, number=2, baud_rate=9600, data_bits = 8,)
station.add_component(ips)
ips.initialise_magnet()


#ips.deinitialise_magnet()
#station.remove_component("IPS")




#Other instruments:


#DMM Agilent Technologies
#dmm = Agilent_34400A("DMM","USB0::0x0957::0x0A07::MY46000282::INSTR") ## Older DMMs (from cryohall)
#station.add_component(dmm)

#DMM Keysight
#dmm_new = Keysight_34465A("DMM_new", "USB0::0x2A8D::0x0101::MY60006526::INSTR") ## New DMMs bought by Andreas
#station.add_component(dmm_new)


#MFLI Zurich Instruments 
#mfli = ziqc.MFLI("mf1i", "dev5564", interface="pcie", host="192.168.86.242") ## All this information can be found in
                                                                            ## In the LabOne user interface
#station.add_component(mfli)





VisaIOError: VI_ERROR_RSRC_BUSY (-1073807246): The resource is valid, but VISA cannot currently access it.

In [5]:
VS = GateParameter(dac, 
                   dac.ch1.volt,
                   name = "VS",
                   unit = "V",
                   val_range = (-0.1, 0.1),
                   scale = 1)

VD = GateParameter(dac,
                   dac.ch2.volt,
                   name = "VD",
                   unit = "V",
                   val_range = (-0.1, 0.1),
                   scale = 1)

VLS = GateParameter(dac, 
                    dac.ch3.volt,
                    name = "VLS",
                    unit = "V",
                    val_range = (-3, 3),
                    scale = 1)

VLD = GateParameter(dac,
                    dac.ch4.volt,
                    name = "VLD",
                    unit = "V",
                    val_range = (-3, 3),
                    scale = 1)

VP = GateParameter(dac,
                   dac.ch5.volt,
                   name = "Vp",
                   unit = "V",
                   val_range = (-3, 3),
                   scale = 1)


VSD = AntisymBiasParameter(name = "VSD",
                              params = (VS, VD),
                              unit = "V")

VLall = VirtualGateParameter(name = "VLall",
                             params = (VLS, VLD),
                             unit = "V",
                             scaling = (1, 1),
                             offset = (0, 0))

# Simultaneous sweeping channels from two different DACs
VBall = VirtualGateParameter(name = "VBall",
                             params = (dac.ch7.volt, dac2.ch1.volt),
                             unit = "V",
                             scaling = (1, 1),
                             offset = (0, 0))

# Specify IV converter gains
gain_SD = ManualParameter('gain_SD', 
                           initial_value = 1e6, 
                           unit = 'V/A')

gain_gates = ManualParameter('gain_gates', 
                              initial_value = 1e9, 
                              unit = 'V/A')

IS = MeasurementParameter(DAQ.ai_0.voltage,
                          name = "I_S",
                          gain = gain_SD,
                          unit="A")

ID = MeasurementParameter(DAQ.ai_1.voltage,
                          name = "I_D",
                          gain = gain_SD,
                          unit="A")

ILS = MeasurementParameter(DAQ.ai_2.voltage,
                           name = "ILS", 
                           gain = gain_gates,
                           unit="A")

ILD = MeasurementParameter(DAQ.ai_3.voltage,
                           name ="ILD",
                           gain = gain_gates,
                           unit = "A")

IP = MeasurementParameter(DAQ.ai_4.voltage,
                          name = "IP",
                          gain = gain_gates,
                          unit = "A")

# needed if you want to measure multiple DAQ channels
ISD = MultiDAQParameter(DAQ,
                        [IS, ID],
                        "ISD",
                        names = ["I_S","I_D"],
                        units = ["A","A"])

Iall = MultiDAQParameter(DAQ,
                         [IS, ID, ILS, ILD, IP],
                         "Iall",
                         names = ["I_S","I_D","ILS","ILD","IP"],
                         units = ["A","A","A","A","A"])



NameError: name 'dac' is not defined

In [ ]:
def detuning(x1,y1,x2,y2):
    slope = (y1-y2)/(x1-x2)
    intercept = (y1+y2-slope*(x1+x2))/2
    det = VirtualGateParameter(name = "det", 
                           params = (VP2, VP1),  
                           set_scaling = (1, params[0]),
                           offsets = (0, params[1]))
    return [slope, intercept]

In [ ]:
#initializing Lock-In parameters 

ZIfreq = GateParameter(uhfli.oscs[0].freq,
                   name = "ZI_freq",
                   unit = "Hz",
                   value_range = (0, 600e6 ),
                   scaling = 1,
                   offset = 0)


ZITC = GateParameter(uhfli.demods[0].timeconstant,
                   name = "ZI_TC",
                   unit = "s",
                   value_range = (0, 10 ),
                   scaling = 1,
                   offset = 0)

#Command to adjust re-zero the ZI phase: ZIPhaseAdjust(1)
ZIPhaseAdjust = GateParameter(uhfli.demods[0].phaseadjust,
                   name = "Phase_adjust",
                   unit = "s",
                   value_range = (0, 1),
                   scaling = 1,
                   offset = 0)


XY = ZILockinParameter(uhfli, ['X','Y'], 
                       'XY', 
                       names = ['X','Y'], 
                       gain = 1,
                       scaling = 1,
                       units = ['V','V'])

XYRPhi = ZILockinParameter(uhfli,
                           ['X','Y','R','Phase'], 
                           'XYRPhi', 
                           names = ['X','Y','R','Phase'], 
                           gain = 1,
                           scaling = 1,
                           units = ['V','V','V','°'])

X = ZILockinParameter(uhfli,
                      ['X'],
                      'X', 
                      names = ['X'], 
                      gain = 1,
                      scaling = 1,
                      units = ['V'])

Y = ZILockinParameter(uhfli,
                      ['Y'],
                      'Y',
                      names = ['Y'], 
                      gain = 1,
                      scaling = 1,
                      units = ['V'])



### IPS-Commands

In [ ]:
ips.heater_on()
ips.heater_off()

In [ ]:
ips._get_field()

In [ ]:
ips.examine()

In [ ]:
ips.local()
ips.remote()

In [ ]:
ips.to_zero()
ips.to_setpoint()

In [ ]:
ips.field_setpoint(0.5)

In [ ]:
ips.deinitialise_magnet()
station.remove_component("IPS")

### IPS tests

In [10]:
ips.heater_on()

Setting switch heater... (wait 40s)


In [17]:
ips.examine()

System Status: 
Normal
Activity: 
To set point
Local/Remote status: 
Local and unlocked
Switch heater: 
On (switch open)
Mode: 
Tesla, Magnet sweep: fast
Polarity: 
Desired: Positive, Magnet: Positive, Commanded: Positive, Both contactors open


In [15]:
ips.field_setpoint(0)

In [16]:
print(ips.field())

0.0


In [14]:
do1d(ips.field_setpoint,0.001,0,5,1,ISD,show_progress = True)

Starting experimental run with id: 225. Using 'qcodes.utils.dataset.doNd.do1d'


  0%|          | 0/5 [00:00<?, ?it/s]

(results #225@C:\Users\ZumAdmin\Documents\GitHub\SPIN_qcodes_stations\phys-dots-40\Rafael\T6A_c5_double_layer.db
 ---------------------------------------------------------------------------------------------------------------
 IPS_field_setpoint - numeric
 ISD - numeric,
 [None],
 [None])

In [18]:
ips.heater_off()

Setting switch heater... (wait 40s)


In [19]:
ips.examine()

System Status: 
Normal
Activity: 
To set point
Local/Remote status: 
Local and unlocked
Switch heater: 
Off magnet at zero (switch closed)
Mode: 
Tesla, Magnet sweep: fast
Polarity: 
Desired: Positive, Magnet: Positive, Commanded: Positive, Both contactors open
